This is normal SNN network without stashing

In [26]:
import snntorch as snn
from snntorch import spikeplot as splt
from snntorch import spikegen

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
import numpy as np
import itertools
import copy

In [27]:
# Hyperparameters
batch_size = 64
epochs = 5
beta = 0.95
# The beta defined here = (1 - delta(t)/Tau)
# Here delta(t) is the clk period
# Tau = Time constant = RC
# This is the approximation of beta = exp(-delta(t)/Tau)
num_steps = 25

In [28]:
# Perform learning over cuda
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [29]:
transform = transforms.Compose([
            transforms.Resize((28, 28)),
            transforms.Grayscale(),
            transforms.ToTensor(),
            transforms.Normalize((0,), (1,))])

mnist_train = datasets.MNIST("/home/neel/Sem_8/RnD/RnD_rough/data", train=True, download=True, transform=transform)
mnist_test = datasets.MNIST("/home/neel/Sem_8/RnD/RnD_rough/data", train=False, download=True, transform=transform)

In [30]:
# Loading the training and test data
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=True, drop_last=True)

In [31]:
# Here we are creating a single hidden layer NN, with 150 nodes
class Net(nn.Module):
    def __init__(self, num_hidden_layer = 150):
        super().__init__()

        # Initialize layers
        self.input_layer = nn.ModuleList([nn.Linear(in_features=784, out_features=1) for i in range(num_hidden_layer)])
        self.lif1 = snn.Leaky(beta=beta)
        self.output_layer = nn.Linear(in_features=num_hidden_layer, out_features=10)
        self.lif2 = snn.Leaky(beta=beta)

    def forward(self, x):

        # Initialize hidden states at t=0
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()

        # Record the final layer
        spk2_rec = []
        mem2_rec = []

        for step in range(num_steps):
            cur1 = [self.input_layer[i](x) for i in range(num_hidden_layer)]
            cur1 = torch.hstack(cur1)
            spk1, mem1 = self.lif1(cur1, mem1)
            cur2 = self.output_layer(spk1)
            spk2, mem2 = self.lif2(cur2, mem2)
            spk2_rec.append(spk2)
            mem2_rec.append(mem2)

        return torch.stack(spk2_rec, dim=0), torch.stack(mem2_rec, dim=0)

# Load the network onto CUDA if available
num_hidden_layer = 150
model = Net(num_hidden_layer=num_hidden_layer).to(device)
next(model.parameters()).is_cuda


True

In [32]:
next(model.parameters()).is_cuda

True

In [33]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [34]:
def accuracy():
    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        for images, label in test_loader:
            images = images.reshape(-1, 28*28).to(device)
            label = label.to(device)

            # Test set forward pass
            test_spk, test_mem = model(images.view(batch_size, -1))
            n_samples += label.size(0)
            _, idx = test_spk.sum(dim=0).max(1)
            n_correct+= (label == idx).sum().item()
            return(n_correct/n_samples * 100)

In [35]:
# Generating non duplicate random nodes to be stashed out of hidden layer per epoch 
import random
stashed = random.sample(range(num_hidden_layer), 15)
stashed

[20, 112, 116, 63, 24, 135, 94, 119, 81, 72, 97, 106, 5, 113, 28]

In [36]:
j=0
n_total_steps = len(train_loader)
for epoch_iter in range(epochs):
    for i,(data, target) in enumerate(train_loader):
        x = data.to(device)
        y = target.to(device)
        spk_rec, mem_rec = model(x.view(batch_size, -1))
        loss_val = torch.zeros((1),device=device)
        for step in range(num_steps):
            loss_val += loss(mem_rec[step], y)
        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()
        if (i+1) % 100 == 0:
             print (f'Epoch [{epoch_iter+1}/{epochs}], Step[{i+1}/{n_total_steps}], Loss: {loss_val.item():.4f}')
             print("-------------------------")
        
        if (i+1) % 300 == 0:
            print("Accuracy is", accuracy())
            for params in model.input_layer[stashed[j]].parameters():
                params.requires_grad = False
            print("Node stashed", stashed[j])
            print("-------------------------")
            j+=1

        

Epoch [1/5], Step[100/937], Loss: 18.5441
-------------------------
Epoch [1/5], Step[200/937], Loss: 21.3070
-------------------------
Epoch [1/5], Step[300/937], Loss: 19.4786
-------------------------
Accuracy is 89.0625
Node stashed 20
-------------------------
Epoch [1/5], Step[400/937], Loss: 11.3613
-------------------------
Epoch [1/5], Step[500/937], Loss: 12.1799
-------------------------
Epoch [1/5], Step[600/937], Loss: 15.0556
-------------------------
Accuracy is 81.25
Node stashed 112
-------------------------
Epoch [1/5], Step[700/937], Loss: 16.1833
-------------------------
Epoch [1/5], Step[800/937], Loss: 17.4335
-------------------------
Epoch [1/5], Step[900/937], Loss: 9.9518
-------------------------
Accuracy is 85.9375
Node stashed 116
-------------------------
Epoch [2/5], Step[100/937], Loss: 11.9394
-------------------------
Epoch [2/5], Step[200/937], Loss: 13.3232
-------------------------
Epoch [2/5], Step[300/937], Loss: 9.5651
-------------------------


In [37]:
# Assertion to see if our nodes have really been stashed even after the training is over
univ = [i for i in range(num_hidden_layer)]
not_stashed = [i for i in univ if i not in stashed]

for i in stashed:
    for j in model.input_layer[i].parameters():
        assert j.requires_grad == False

for i in not_stashed:
    for j in model.input_layer[i].parameters():
        assert j.requires_grad == True

In [38]:
accuracy()

95.3125

In [39]:
# #Creating computational graph to see, whether all the neurons are contributing or not
# batch = next(iter(train_loader))
# yhat = net(batch[0].reshape(-1,28*28).to(device)) # Give dummy batch to forward().

# from torchviz import make_dot

# make_dot(yhat, params=dict(list(net.named_parameters()))).render("trial_1", format="png")